In [1]:
!pip install transformers langchain huggingface_hub bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 794.3/794.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 2.2 MB/s eta 0:00:00


In [2]:
import torch
import sys
import os
from tqdm import tqdm
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain, MapReduceDocumentsChain, ReduceDocumentsChain, StuffDocumentsChain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from tqdm import tqdm
import transformers
import pandas as pd
import numpy as np
import logging
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from huggingface_hub import hf_hub_download
from langchain.llms import HuggingFacePipeline
from torch import  bfloat16
import transformers

In [3]:
def load_intel_model():
    sample_generation = True
    model_name='bigscience/bloom'

    #cache_dir='/srv/ailab_data/hf_cache'

    tokenizer = transformers.AutoTokenizer.from_pretrained(
        pretrained_model_name_or_path=model_name,
    )
    bnb_config = transformers.BitsAndBytesConfig(
        load_in_8bit=False,
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=bfloat16
    )
    model_config = transformers.AutoConfig.from_pretrained(
        pretrained_model_name_or_path=model_name,
    )
    model = transformers.AutoModelForCausalLM.from_pretrained(
        pretrained_model_name_or_path=model_name,
        resume_download=True,
        trust_remote_code=True,
        config=model_config,
        quantization_config=bnb_config,
        device_map='auto'
    )
    model.eval()
    pipeline_args = {}
    if sample_generation:
        pipeline_args['top_k'] = 20
        pipeline_args['top_p'] = 0.95
        pipeline_args['temperature'] = 0.1
    generate_text = transformers.pipeline(
        model=model,
        tokenizer=tokenizer,
        return_full_text=True,
        task='text-generation',
        do_sample=sample_generation,
        max_new_tokens=4095,
        repetition_penalty=1.1,
        **pipeline_args
    )
    generate_text.model.config.pad_token_id = generate_text.model.config.eos_token_id

    llm = HuggingFacePipeline(pipeline=generate_text)

    return llm

In [4]:
llm = load_intel_model()

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/573 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/63.7k [00:00<?, ?B/s]

...

model_00009-of-00072.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model_00010-of-00072.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:992: UserWarning: Not enough free disk space to download the file. The expected file size is: 4932.88 MB. The target location /root/.cache/huggingface/hub/models--bigscience--bloom/blobs only has 3775.43 MB free disk space.
  warnings.warn(


model_00011-of-00072.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

OSError: ignored

In [ ]:
CV_Candidate = """Aleksandr Shirshov
alexnder.shirshov@gmail.com +7 987 675-07-91
About me
Age: 23
I may not know something, but give me Google and a couple of hours and I'll figure it out. I am not afraid to learn new things and effectively assimilate new information. My 3+ years experience in consulting and entrepreneurship and my technical background will make me a valuable employee for any company.
  Education
Higher School of Economics (2017-2021) Bachelor. Faculty of Physics.
Solid State Physics. GPA: 8 out of 10
Work Experience
Quantum Brains August 2023 – Until now
ML Engineer / AI LAB
ITMO (2023-2025)
Master. Artificial Intelligence. AI Talent Hub
     • Conducting research to solve a problem in NLP, CV using SOTA approaches
• Development of a financial chatbot using LLM agents (Lungchain, Llama
index, BERT, GPT)
• Analysis of news sentiment to develop an optimal trading strategy for the RL
agent
Moscow Construction Department April 2023 – August 2023 Data Scientist
• Design and prepare reports using PowerBI
• Create and configure DWH to manage data in the company
• Creation of ETL processes and data processing pipelines
• Developed a service for forecasting the construction time of an object
• Developed a smart camera service at a construction site using CV and product
deployment
Constanta August 2021 - April 2023 Consultant, Management Consulting
• Lead a project to create a DWH and PowerBI reports
• Work on strategic consulting projects in the field of Telecom, IT, AI, Industry • Market assessment for strategic initiatives
• Development of a target operating model for factories
• Search, analysis, structuring of information for visualization on MS PP slides • Participation in client meetings on the company's strategy
CEO&Founder December 2020 – August 2023 Entrepreneur, Company Euromed
• Together with the team, we created our own production of recirculators - a device for cleaning and disinfecting air
• With an investment of 300,000 rubles, the company's turnover reached 4,000,000 rubles
• Continuous participation in public procurement, winning percentage 56%
• Created a multifunctional universal product at an affordable price
• Responsible for the stages of production, lead a team of 4 people
KPMG January 2020 – January 2021
Сonsultant 1 year. Management Consulting/IT Audit.
• Communication with the client and identification of risks in the framework of an IT audit • Worked on large projects as part of an audit of financial statements
• Created a new procedure for testing IT Systems
• Processed data via SQL and MS Excel ACHIEVEMENTS
1st place ITMO hackathon bootcamp 2023 Created a generative AI service for text and photo generation (Llama, StableDiffusion) for marketplace sellers
2nd place LCT hackathon (the largest hackathon in the Russian Federation) 2023 Created a Сomputer Vision platform for calculating the readiness of finishing in residential buildings
Diploma of the 1st degree Olympiad "MIIT" 2017
All-Russian Physics Olympiad from MIIT
Diploma of the 3rd degree Olympiad "PHIZTECH" 2017
All-Russian Olympiad in Physics from MIPT
ADDITIONAL INFORMATION
 Language skills: Prof. skills:
Courses:
English (B2)
Python (Pandas, NumPy, scipy, matplotlib, seaborn, scikit-learn, pytorch, FastAPI, catboost, XGBoost, LightGBM, YOLO, Streamlit), AirFlow SQL (PostgresSQL), PowerBI, DataLense, MS Office.
Karpov.courses: StartML (2023)
Data Camp: Data Scientist with Python (2022)
Higher School of Economics: Theory of Organization and HR management, Marketing, Project management, Strategic planning."""

In [ ]:
Vacancy = """Machine Learning Operations Engineer (All genders)

Company Description

For over 20 years, zooplus has been at the forefront of the pet category in eCommerce, standing as the Number 1 online e-commerce Pet Platform. Now, we are on the cusp of an exhilarating new frontier, venturing into the world of Generative AI.

Job Description

Zooplus has been at the forefront of data collection for over two decades, continuously expanding our capabilities. As a Machine Learning Operations Engineer in the Data & Analytics department at Zooplus, you'll be part of an exciting journey into the uncharted territory of Generative AI (Gen AI). We're committed to making remarkable progress in this field with your expertise.

This position is one of several roles we are currently recruiting for, in the ML Ops Engineer area. As a member of this greenfield team, you will play a crucial role in the new Gen AI team we are establishing within the Data & Analytics department. Your responsibilities will include collaborating with the Product Owner, R&D, and the Data Scientist team, as well as providing support to business AI consultants for prototyping solutions and managing backlogs.

Key Responsibilities:
Data Exploration: Delve into our growing set of external and internal data flows to generate insightful analyses that drive innovation throughout Zooplus.
Machine Learning: Research, design, validate, and deploy machine learning models using structured and unstructured data, employing advanced AI techniques.
Communication: Effectively communicate results and conclusions to business stakeholders through data visualizations.
Agile Collaboration: Work in an international, agile team to build models, applications, and solutions that make Zooplus smarter and more adaptable in the dynamic e-commerce market.
Our tech stack includes but not limited to:

Proficiency in Python and Spark.
Experience with AWS and Terraform for cloud infrastructure.
Git and Jenkins for version control and CI/CD.
Qualifications
Required Skills:

Excellent team player with a "can do" attitude.
MS/PhD in Computer Science or a related field, specializing in ML/AI.
Good understanding of statistical and AI/ML methods, providing expertise in the design, development, implementation, and testing of AI/ML solutions.
Work experience using Python and Spark (or equivalent) to create big data applications and analysis.
Knowledge of relational databases and query authoring (SQL).
Working level English Proficiency
Desired but not required:

Experience in solving optimization problems.
Experience working in cloud environments. Desirable AWS (Kinesis, Lambda, Redshift, RDS, Athena, EC2, EMR, S3, EMR, Sagemaker).
CI/CD experience.
Java (with Spring Framework).
Experience with Atlassian platform (JIRA, Bitbucket, Confluence, etc).
Additional Information
With more than 1,000 passionate professionals located across 10 European offices, we believe our success comes from working together and leveraging our international strengths. Expect to work in a hybrid environment, collaborating with colleagues in different locations remotely or face-to-face at the office.

Our benefits:

20% discount in our zooplus shop
Internal and external training
Team events
28 vacation days and days off on 24th and 31st of December
Corporate rates at a local gym chain (Body & Soul)
Company mobile phone for work and personal use"""

## Defining candidates's CV prompt

In [ ]:
LEVEL_PROMPT = """You are an experienced HR at a large technology company, you give advice to candidates for the position of software developer.
For the candidate's resume, which will be highlighted with triple backquotes You need to define the professional level of the candidate, you have to choose one of [Intern, Junior, Junior+, Middle, Middle+, Senior, Senior+, Team Lead] and give answer in one sentences.

Candidate's resume:
```{text}```

Helpful answer:
"""

EXP_PROMPT = """You are an experienced HR at a large technology company, you give advice to candidates for the position of software developer.
For the candidate's resume, which will be highlighted with triple backquotes You need to define the information about the candidate's jobs experience. You should to describe it if possible: company name, position, hours of work, summarize information about work desrpition.
Please provide results as a key points for each candidate's job experience.


Candidate's resume:
```{text}```

Helpful answer:
"""

SKILLS_PROMPT = """You are an experienced HR at a large technology company, you give advice to candidates for the position of software developer.
For the candidate's resume, which will be highlighted with triple backquotes You need to define the candidate's hard skills and soft skills, if they are exists in resume.
Please provide results as a list without professions in it



Candidate's resume:
```{text}```

Helpful answer:
"""


EDU_PROMPT = """You are an experienced HR at a large technology company, you give advice to candidates for the position of software developer.
For the candidate's resume, which will be highlighted with triple backquotes you need to defune the candidate's "universities", "refresher courses", "olympiades and hackathons" (if this is included in the text)
Please provide results with name of educational institution and specialty.


Candidate's resume:
```{text}```

Helpful answer:

"""

In [ ]:
agent_prompts = [LEVEL_PROMPT, EXP_PROMPT, SKILLS_PROMPT, EDU_PROMPT]

mapping = {
    1: 'grade',
    2: 'experience',
    3: 'skills',
    4: 'education'
}

In [ ]:
candidate_dict = {}

for i, prompt_part in enumerate(tqdm(agent_prompts)):
  prompt = PromptTemplate(
      input_variables=['texts'],
      template=prompt_part
  )
  chain = LLMChain(llm=llm, prompt=prompt)
  response = chain.run(CV_Candidate)
  candidate_dict[i] = response

In [ ]:
candidate_info = (f"LEVEL:\n{candidate_dict[0]}\n"
                 f"EXPERIENCE:\n{candidate_dict[1]}\n"
                 f"SKILLS:\n{candidate_dict[2]}\n"
                 f"EDUCATION:\n{candidate_dict[3]}")

# Now you can print the combined_info variable
print(candidate_info)

## Defining prompt for vacancy description

In [ ]:
KEY_VACANCY_POMPT = """You are an experienced HR at a large technology company,you hired a lot of great IT specialists.
For the presented vacancy description, highlight key information in triple backquotes, which may contain requirements for the vacancy, relevant experience, level of the candidate and other useful information.
Please provide information by topics and be sure to determine the approximate professional level of a potential candidate for a vacancy (Intern, Junior, Junior+, Middle, Middle+, Senior, Senior+, Team Lead).

Vacancy description:
```{text}```

Helpful answer:

"""

In [ ]:
prompt = PromptTemplate(
      input_variables=['texts'],
      template=KEY_VACANCY_POMPT
  )
chain = LLMChain(llm=llm, prompt=prompt)
key_vacancy = chain.run(Vacancy)
print(key_vacancy)


## Defining recommendations

In [ ]:
RECS_PROMPT = """You are an experienced HR at a large technology company,you hired a lot of great IT specialists.
Below you will find information about the candidate, which includes his professional level, experience, skills, and education.
A description of the vacancy for which the candidate is being selected will also be provided.

Your task is to give a recommendation to the candidate for the vacancy, which should include:
- general relevance of the vacancy
- recommendation
- recommendations for providing additional information in a resume based on skills
- recommendations for specifying additional skills


Candidate's resume:
{resume}

Vacancy description:
{vacancy}

Helpful answer:
"""

In [ ]:
SCORE_PROMPT = """You are an experienced HR at a large technology company,you hired a lot of great IT specialists.
Below you will find information about the candidate and recommendations to resume, which includes his professional level, experience, skills, and education.
A description of the vacancy for which the candidate is being selected will also be provided.

Your task is to evaluate the suitability of the resume for the vacancy from 0 to 100, where 0 is completely unmatched and 100 is perfect match. When assessing, be sure to take into account the candidate’s grade, technical and the grade for which they are looking for in the vacancy
Provide an answer is just score from 0 to 100.

Candidate's resume:
{resume}

Recommendations for resume:
{recs}

Vacancy description:
{vacancy}

Helpful answer:
"""

In [ ]:
prompt = PromptTemplate(
      input_variables=["resume", "recs","vacancy"],
      template=RECS_PROMPT
  )
chain = LLMChain(llm=llm, prompt=prompt)
recs = chain.run({'resume': candidate_info, 'vacancy': key_vacancy})
print(recs)

In [ ]:
prompt = PromptTemplate(
      input_variables=["resume", "recs",  "vacancy"],
      template=SCORE_PROMPT
  )
chain = LLMChain(llm=llm, prompt=prompt)
score = chain.run({'resume': candidate_info, 'recs': recs, 'vacancy': key_vacancy})
print(score)